<a href="https://colab.research.google.com/github/rithwikbabu/financial-models/blob/main/binomial_tree_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np

In [3]:
class BinomialTreeModel:
    """
    This class implements the Binomial Tree model for pricing American options.
    """

    def __init__(self, steps=100):
        """
        Initializes the BinomialTreeModel class.
        """
        self.steps = steps

    def price_option(self, S, K, T, r, sigma, option_type='call'):
        """
        Prices an American option (call or put) using the Binomial Tree model.
        """
        dt = T / self.steps
        u = np.exp(sigma * np.sqrt(dt))
        d = 1 / u
        p = (np.exp(r * dt) - d) / (u - d)

        # init asset
        asset_prices = np.asarray([S * (u ** j) * (d ** (self.steps - j)) for j in range(self.steps + 1)])

        # init options
        if option_type == 'call':
            option_values = np.maximum(asset_prices - K, 0)
        elif option_type == 'put':
            option_values = np.maximum(K - asset_prices, 0)
        else:
            raise ValueError("Invalid option type. Use 'call' or 'put'.")

        # bottom up
        for i in range(self.steps, 0, -1):
            option_values = np.exp(-r * dt) * (p * option_values[1:] + (1 - p) * option_values[:-1])

            # check for early exercise
            if option_type == 'call':
                asset_prices = asset_prices[:-1] * u  # adj to prev step
                option_values = np.maximum(option_values, asset_prices - K)
            elif option_type == 'put':
                asset_prices = asset_prices[:-1] * u  # adj to prev step
                option_values = np.maximum(option_values, K - asset_prices)

        return option_values[0]

In [4]:
model = BinomialTreeModel(steps=500)
price_call = model.price_option(S=100, K=100, T=1, r=0.05, sigma=0.2, option_type='call')
price_put = model.price_option(S=100, K=100, T=1, r=0.05, sigma=0.2, option_type='put')

print(f"American Call Option Price: {price_call}")
print(f"American Put Option Price: {price_put}")

American Call Option Price: 10.44658513644654
American Put Option Price: 6.088810110702773
